In [1]:
# importando bibliotecas
import gurobipy as gp
from gurobipy import GRB

Considere o seguinte MIP.

$
\begin{align*}
\max \ & x +   y + 2 z \\
& x + 2 y + 3 z \leq 4 \\
& x +   y      \geq 1 \\
& x, y, z \in \mathbb{B} \\
\end{align*}
$

Implemente o modelo usando API gurobipy.

In [3]:
# criando um novo modelo
m0 = gp.Model("mip")

# criando variáveis
x = m0.addVar(vtype=GRB.BINARY, name="x")
y = m0.addVar(vtype=GRB.BINARY, name="y")
z = m0.addVar(vtype=GRB.BINARY, name="z")

# definindo a função objetivo
m0.setObjective(x + y + 2 * z, GRB.MAXIMIZE)

# adicionando a restrição, x + 2 y + 3 z <= 4
m0.addConstr(x + 2 * y + 3 * z <= 4, "restrição 0")

# adicionando a restrição, x + y >= 1
m0.addConstr(x + y >= 1, "restrição 1")
    
# desabilitando o log
m0.params.outputflag = 0
    
m0.write("mip0.mps")

# resolvendo o modelo
m0.optimize()

# imprimindo a solução ótima
print('Obj = %g' % m0.objVal)

# imprimindo as soluções
for v in m0.getVars():
    print('%s = %g' % (v.varName, v.x))

Academic license - for non-commercial use only - expires 2022-09-29
Using license file /opt/gurobi912/gurobi.lic
Obj = 3
x = 1
y = 0
z = 1


Implemente o modelo usando a idéia de matriz.

In [ ]:
#!pip3 install scipy

In [4]:
import numpy as np
import scipy.sparse as sp

# criando um novo modelo
m1 = gp.Model("matrix")

# criando as variáveis
x = m1.addMVar(shape=3, vtype=GRB.BINARY, name="x")

# definindo a função objetivo
obj = np.array([1.0, 1.0, 2.0])
m1.setObjective(obj @ x, GRB.MAXIMIZE)

# construindo a matriz de restrição, esparsa
val = np.array([1.0, 2.0, 3.0, -1.0, -1.0])
row = np.array([0, 0, 0, 1, 1])
col = np.array([0, 1, 2, 0, 1])

A = sp.csr_matrix((val, (row, col)), shape=(2, 3))

# construindo o vetor do lado direito, rhs
rhs = np.array([4.0, -1.0])

# adicionando as restrições
m1.addConstr(A @ x <= rhs, name="c")
    
# gerando .lp
m1.write("matrix.lp")

# resolvendo o problema
m1.optimize()

# imprimindo a solução ótima
print(x.X)
    
# imprimindo o valor ótimo
print('Obj = %g' % m1.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x8d4960d3
Variable types: 0 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.03 seconds
Thread count was 1 (of 8 available processors)

Solution count 2: 3 2 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
[1. 0. 1.]
Obj = 3


# Capacitated facility location problem

Uma companhia entrega diariamente seus produtos que são produzidos em 5 fábricas para 4 armazens. Nesse processo de entrega existem dois tipos de custos, o custo associado ao transporte e o custo fixo de uso da fábrica. Cada fábrica possui um limite quanto a quantidade de produtos a serem entregues. A companhia estudar fechar algumas fábricas para reduzir custos. Quais fábricas a companhia deve fechar, com a finalidade de reduzir os custos?

In [5]:
# demanda dos armazéns
demand = [15, 18, 14, 20]

# capacidade de produção da cada fábrica
capacity = [20, 22, 17, 19, 18]

# custo fixo de cada fábrica
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# custo de transporte
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# quantidade de fábricas
plants = range(len(capacity))

# quantidade de armazéns
warehouses = range(len(demand))

# criando modelo
m2 = gp.Model("cfl")

# y[p]: y[p] == 1 se a fábrica p é aberta, 0 caso contrário.
y = m2.addVars(plants,vtype=GRB.BINARY,obj=fixedCosts,name="y")

# x[w,p]: quantidade transportada da fábrica p para o armazém i
x = m2.addVars(warehouses, plants, obj=transCosts, name="x")

# sentido da função objetivo
m2.modelSense = GRB.MINIMIZE

# restrição de produção
m2.addConstrs((x.sum('*', p) <= capacity[p] * y[p] for p in plants), "capacidade")

# restrição de demanda
m2.addConstrs((x.sum(w,'*') == demand[w] for w in warehouses), "demanda")

# salva o modelo
m2.write('cfl.lp')

# usa o método barrier para resolver o relaxação linear na raiz
m2.Params.method = 2

# resolve o problema
m2.optimize()

# imprime a solução
print('\nCusto total: %g' % m2.objVal)

print('\n Solução:')
for p in plants:
    if y[p].x > 0.99:
        print('\nFábrica %s é aberto' % p)
        for w in warehouses:
            if x[w, p].x > 0:
                print('Transporte de %g unidades para o armazém %s' % (x[w, p].x, w))
    else:
        print('\nFábrica %s é fechada!' % p)

Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0xb5a24dbe
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.94290841e+05 -2.24842916e+05  7.25e+00 3.75e+03  

In [6]:
# demanda dos armazens
demand = [15, 18, 14, 20]

# capacidade de produção das fábricas
capacity = [20, 22, 17, 19, 18]

# custo fixo de abertura das fábricas
fixedCosts = [12000, 15000, 17000, 13000, 16000]

# custo de transporte
transCosts = [[4000, 2000, 3000, 2500, 4500],
              [2500, 2600, 3400, 3000, 4000],
              [1200, 1800, 2600, 4100, 3000],
              [2200, 2600, 3100, 3700, 3200]]

# quantidade de fábricas
plants = range(len(capacity))

# quantidade de armazéns
warehouses = range(len(demand))

# criando modelo
m3 = gp.Model("capfacility1")

# You could use Python looping constructs and m.addVar() to create
# these decision variables instead.  The following would be equivalent
# to the preceding two statements...
#
# variável y: y[p] == 1 se a fábrica p é aberta, 0 caso contrário
y = []
for p in plants:
    y.append(m3.addVar(vtype=GRB.BINARY,obj=fixedCosts[p],name="y[%d]" % p))

# variável x: x[w,p] é quantidade transportada da fábrica p para o armazém w
x = []
for w in warehouses:
    x.append([])
    for p in plants:
        x[w].append(m3.addVar(obj=transCosts[w][p],name="x[%d,%d]" % (w, p)))

# problema de mininização
m3.modelSense = GRB.MINIMIZE

# restrição de produção
for p in plants:
    m3.addConstr(sum(x[w][p] for w in warehouses) <= 
                 capacity[p] * y[p], "Capacidade[%d]" % p)

# restrição de demanda
for w in warehouses:
    m3.addConstr(sum(x[w][p] for p in plants) == demand[w], "Demanda[%d]" % w)

# escrevendo o modelo
m3.write('facility1.lp')

# usando o método barreira para resolver a relaxação na raiz
m3.Params.method = 2

# resolve o problema
m3.optimize()

# imprime a solução
print('\nCusto total: %g' % m3.objVal)
print('\n Solução:')
for p in plants:
    if y[p].x > 0.99:
        print('\nFábrica %s é aberto' % p)
        for w in warehouses:
            if x[w][p].x > 0:
                print('Transporte de %g unidades para o armazém %s' % (x[w][p].x, w))
    else:
        print('\nFábrica %s é fechada!' % p)

Changed value of parameter method to 2
   Prev: -1  Min: -1  Max: 5  Default: -1
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 9 rows, 25 columns and 45 nonzeros
Model fingerprint: 0xb5a24dbe
Variable types: 20 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+03, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 2e+01]
Presolve time: 0.00s
Presolved: 9 rows, 25 columns, 45 nonzeros
Variable types: 20 continuous, 5 integer (5 binary)
Root barrier log...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.000e+01
 Factor NZ  : 4.500e+01
 Factor Ops : 2.850e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.94290841e+05 -2.24842916e+05  7.25e+00 3.75e+03  

# Uncapacitated facility location problem

Considere a formulação do facility location problem apresentado em sala de aula.

$
\begin{align*}
\min \ & \sum_{i \in M} \sum_{j \in N} c_{ij} x_{ij} + \sum_{j \in N} f_j y_j \\
& \sum_{j \in N} x_{ij} = 1 \ \mbox{ for } i=1, \ldots, m \\
& \sum_{i \in M} x_{ij} \leq m \cdot y_j \mbox{ for } j \in N \\
& x_{ij} \geq 0 \mbox{ for } i \in M, j \in N \\
& y_j \in \{ 0, 1 \} \mbox{ for } j \in N
\end{align*}
$

Resolva uma instância do uncapacitated facility problem onde $f_j$ é o custo do depósito $j$ aberto, e $c_{ij}$ é o custo da satisfação da demanda de todos os clientes $i$'s pelo depósito $j$, com 

* número de clientes: 6

* número de depósitos: 5

* custos fixo:

$
f = [4,3,4,4,7]
$

* custo de transporte:

$
c = 
\left[
\begin{array}{ccccc}
12 & 13 & 6 & 0 & 1 \\
8 & 4 & 9 & 1 & 2 \\
2 & 6 & 6 & 0 & 1 \\
3 & 5 & 2 & 1 & 8 \\
8 & 0 & 5 & 10 & 8 \\
2 & 0 & 3 & 4 & 1
\end{array}
\right]
$

Fonte: 

In [7]:
# dimensões
m = 6
n = 5

# custos fixo
f = [4, 3, 4, 4, 7]

# custo de transporte
c = [[12, 13, 6, 0, 1],
     [8, 4, 9, 1, 2],
     [2, 6, 6, 0, 1],
     [3, 5, 2, 1, 8],
     [8, 0, 5, 10, 8],
     [2, 0, 3, 4, 1]]

# conjuntos
N = range(n)
M = range(m)

# criando o modelo
m4 = gp.Model("ufl")

# definindo a variável binária y
y = m4.addVars(n,vtype=GRB.BINARY,obj=f,name="y")

# definindo a variável de produção x
x = m4.addVars(m,n,obj=c,name="x")

# restrição de demanda
m4.addConstrs((x.sum(i) == 1 for i in M), "demanda")

# link entre y e x
m4.addConstrs((x.sum('*', j) <= m*y[j] for j in N ), "limite")

# problema de minimização
m4.modelSense = GRB.MINIMIZE

# escrevendo o modelo
m4.write('facility.lp')

# resolvendo o problema
m4.optimize()

# imprimindo o valor ótimo
print('\nSolução ótima: %g \n' % m4.objVal)

# imprimindo a solução ótima
for j in m4.getVars():
    if j.x > 0:
        print('%s = %g' % (j.varName, j.x))



Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 11 rows, 35 columns and 65 nonzeros
Model fingerprint: 0x8463997b
Variable types: 30 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve time: 0.00s
Presolved: 11 rows, 35 columns, 65 nonzeros
Variable types: 30 continuous, 5 integer (5 binary)

Root relaxation: objective 5.666667e+00, 2 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    5.66667    0    2          -    5.66667      -     -    0s
H    0     0                       9.0000000    5.66667  37.0%     -    0s

Explored 1 nodes (2 simplex iterations) in 0.01 seconds
Thread count was 8 (of